In [1]:
import pandas as pd
import numpy as np
import pickle

# from tqdm import tqdm
# tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict
import copy
from sklearn.model_selection import train_test_split


from util import *
import evaluate_auth

# from functools import partial
# from tqdm import tqdm
# tqdm = partial(tqdm, position=0, leave=True)

In [2]:
import torch
import importlib
import logging
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import yaml
from models_cnn import *

# _log = logging.getLogger("train_cnn")
# logging.basicConfig(level=logging.INFO)

In [3]:
from datetime import datetime

time_begin = datetime.now()
print("Begining Time:\n", time_begin)

Begining Time:
 2024-02-27 00:33:55.592921


In [4]:
DATA_ADDRESS = "./data"
# os.listdir(DATA_ADDRESS)
# TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
TEST_DIR = os.path.join(DATA_ADDRESS,'preprocessed','test')
# os.listdir(TRAIN_DIR)

output_dir = "./model/auth/cnn"
if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [5]:
X,y = load_data(
    dir_feature = TEST_DIR,
    file_prefix = 'mfcc_len5_fft2048_mels128_mfcc17_',
    dir_df_index = os.path.join(DATA_ADDRESS,'df_index_test.pkl'),
    n_interval=500,
    flatten=False)

100%|██████████| 50/50 [00:00<00:00, 104.84it/s]


Loaded np array
 with X shape: (24800, 17, 216)
 with y shape: (24800,)


In [6]:
# CNN
PATH = "model/basecnn200"
model_set = torch.load(f"{PATH}/model.pt")
clf_pre = model_set["model"]

In [7]:
clf_pre

BaseCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 5), stride=(1, 2), padding=(1, 2))
  (pool1): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 5), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=200, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [8]:
params_path = 'params.yml'
with open(params_path, 'r') as file:
    params = yaml.safe_load(file)
params

algo = params["algorithm"]
batch_size = int(algo["params"]["batch_size"])
n_epochs = int(algo["params"]["epochs"])
patience = int(algo["params"]["patience"])
learning_rate = float(algo["params"]["learning_rate"])

model_save_path = os.path.join(output_dir,'cnn_all_auth.py')

In [9]:
# y_binary = np.where(y == 2137., 1, 0)

# # Step 2
# X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42,stratify=y_binary)

# train_dataset = VoiceDataset(X_train, y_train)
# val_dataset = VoiceDataset(X_val, y_val)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)




In [10]:

# # Step 3
# device = get_cuda_device()
# clf = copy.deepcopy(clf_pre)
# clf = clf.to(device)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(clf.parameters(), lr=learning_rate)

# # _log.info("Training model...")
# clf.fit(train_loader,
#               val_loader,
#               criterion,
#               optimizer,
#               device,
#               n_epochs,
#               patience,
#               model_save_path)
# # _log.info(f"Model saved at {model_save_path}")

In [11]:
# len(y_val)

In [12]:
# pred_list, label_list = clf.predict(val_loader, device)

In [13]:
# y_test_pred = np.concatenate(pred_list)
# y_test_pred

In [14]:
# label_list

In [15]:
# y_val

In [16]:
# np.unique(y_val, return_counts=True)

In [17]:
# y_val_true = np.concatenate(label_list)
# y_val_true

In [18]:
# np.unique(y_val_true, return_counts=True)

In [19]:
# auth with cnn
device = get_cuda_device()

dict_evaluation = defaultdict(dict)
# Step 1
unique_speakers = np.unique(y)

for speaker_id in tqdm(unique_speakers, desc="Speaker"):
    y_binary = np.where(y == speaker_id, 1, 0)

    # Step 2
    X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42,stratify=y_binary)

    train_dataset = VoiceDataset(X_train, y_train)
    val_dataset = VoiceDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    
    # Step 3 - Fine Tuning
    
    clf = copy.deepcopy(clf_pre)
    clf = clf.to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(clf.parameters(), lr=learning_rate)

    clf.fit(train_loader,
                  val_loader,
                  criterion,
                  optimizer,
                  device,
                  n_epochs,
                  patience,
                  model_save_path)


    # Step 4 - Evaluation
    train_pred_list, train_label_list = clf.predict(train_loader, device)
    y_train_pred = np.concatenate(train_pred_list)
    y_train_true = np.concatenate(train_label_list)
    val_pred_list, val_label_list = clf.predict(val_loader, device)
    y_val_pred = np.concatenate(val_pred_list)
    y_val_true = np.concatenate(val_label_list)

    dict_evaluation['train'][speaker_id] = evaluate_auth.evaluate_model(y_train_true,y_train_pred)
    dict_evaluation['test'][speaker_id] = evaluate_auth.evaluate_model(y_val_true,y_val_pred)



Speaker:   0%|          | 0/100 [00:00<?, ?it/s]

Speaker: 100%|██████████| 100/100 [1:50:15<00:00, 66.16s/it]


In [20]:
dict_evaluation

defaultdict(dict,
            {'train': {115.0: {'eer': 0.014423076923076936,
               'frr': 0.014634146341463415,
               'far': 0.0,
               'acc': 0.9998487903225807,
               'recall': 0.9853658536585366,
               'precision': 1.0,
               'f1': 0.9926289926289926},
              118.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              154.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              166.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              175.0: {'eer': 0.004854368932038839,
               'frr': 0

In [21]:
evaluate_auth.evaluation_to_df(dict_evaluation)


,eer,frr,far,acc,recall,precision,f1
train,0.004699,0.004903,0.000004,0.999947,0.995097,0.99959,0.997289
test,0.052427,0.056949,0.000002,0.999425,0.943051,0.99980,0.970104


In [22]:
time_end = datetime.now()
print("End Time:\n", time_end)
print("Run Time:\n", time_end-time_begin)

End Time:
 2024-02-27 02:24:16.828345
Run Time:
 1:50:21.235424


baseCNN

out put 1

source stratified split

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>far</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.264644</td>
      <td>0.401333</td>
      <td>0.000000</td>
      <td>0.996177</td>
      <td>0.598667</td>
      <td>0.96</td>
      <td>0.713303</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.466716</td>
      <td>0.930000</td>
      <td>0.000196</td>
      <td>0.988903</td>
      <td>0.070000</td>
      <td>0.08</td>
      <td>0.073333</td>
    </tr>
  </tbody>
</table>
</div>

output 2

all stratified split

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>far</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.004699</td>
      <td>0.004903</td>
      <td>0.000004</td>
      <td>0.999947</td>
      <td>0.995097</td>
      <td>0.99959</td>
      <td>0.997289</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.052427</td>
      <td>0.056949</td>
      <td>0.000002</td>
      <td>0.999425</td>
      <td>0.943051</td>
      <td>0.99980</td>
      <td>0.970104</td>
    </tr>
  </tbody>
</table>
</div>